In [1]:
import sqlalchemy
import pandas as pd
from sqlalchemy.orm import sessionmaker
import requests
import json
from datetime import datetime
import datetime
import sqlite3

DATABASE_LOCATION = "sqlite:///my_played_tracks.sqlite"
USER_ID = "22rs5vo6j2olnut7leymyzn6q"
TOKEN = "BQCC8ZqM4i8FElaZ7k7go-HJbgaYV6vTov6_OcjP6NNbsNrrjs-wHA-BxzS3dVz250UcZPb1QvxRBn80C1zXRNSIf712JkbAycg0T-Gu6Ymp1H1-vFosqvjWLW4ONnr9W9xpsIfxkDwpK2y_tTLxLp7VfMsulRSt3PjmsXmB-nrVcMuK3egYv7NUR6pMLPgfl8IfGg"

headers = {
    "Accept" : "application/json",
    "Content-Type":"application/json",
    "Authorization":"Bearer {token}".format(token=TOKEN)
}

today = datetime.datetime.now()
yesterday = today - datetime.timedelta(days=1)
yesterday_unix_timestamp = int(yesterday.timestamp()) * 1000

In [66]:
r = requests.get("https://api.spotify.com/v1/me/player/recently-played?limit=50&after={time}".format(\
    time=yesterday_unix_timestamp), headers=headers)

data = r.json()
data_df = pd.json_normalize(data['items'])



In [67]:
data_columns = ["played_at",
                'track.album.artists',
                "track.name",
                "track.popularity"]

data_df = data_df[data_columns]
data_df['track.album.artists'] = data_df['track.album.artists'].str[0].str['name']
data_df.head(5)

,played_at,track.album.artists,track.name,track.popularity
0,2023-03-21T22:54:08.614Z,Saib,When It Rains,51
1,2023-03-21T22:51:35.042Z,Tom Doolie,New Fields,55
2,2023-03-21T22:49:21.944Z,Saib,Sakura Trees,60
3,2023-03-21T14:12:39.157Z,Sufjan Stevens,Futile Devices,59
4,2023-03-21T13:54:15.957Z,Conor Oberst,Cape Canaveral,26


In [68]:
column_names = {
    'played_at':'played_at',
    'track.album.artists':'artist',
    'track.name':'track_name',
    'track.popularity':'popularity'
}

data_df.rename(columns=column_names, inplace=True)


In [73]:
data_df.sort_values('popularity',ascending=False)

,played_at,artist,track_name,popularity
21,2023-03-21T12:34:13.356Z,Sufjan Stevens,Mystery of Love,69
24,2023-03-21T12:14:43.902Z,Unknown Mortal Orchestra,So Good At Being in Trouble,65
20,2023-03-21T12:38:21.045Z,Various Artists,Visions of Gideon,65
19,2023-03-21T12:42:21.471Z,Sufjan Stevens,Death with Dignity,63
17,2023-03-21T12:49:53.947Z,Sufjan Stevens,The Only Thing,61
7,2023-03-21T13:29:58.196Z,Oscar Peterson,Blue And Sentimental,61
2,2023-03-21T22:49:21.944Z,Saib,Sakura Trees,60
18,2023-03-21T12:45:10.175Z,Sufjan Stevens,To Be Alone With You,59
16,2023-03-21T12:55:59.457Z,Sufjan Stevens,Chicago,59
22,2023-03-21T12:29:53.717Z,Sufjan Stevens,Futile Devices,59


In [76]:
data_df.artist.value_counts()

Sufjan Stevens              15
Belle and Sebastian          6
Saib                         2
Oscar Peterson               2
Tom Doolie                   1
Conor Oberst                 1
Live Foyn Friis              1
The Roots                    1
Various Artists              1
Unknown Mortal Orchestra     1
Name: artist, dtype: int64

In [77]:
print(f'You have listened {data_df.artist.value_counts().shape[0]} artist recently')

You have listened 10 artist recently


In [78]:
def data_validation(df: pd.DataFrame) -> bool:
    #Check if dataframe is empty
    if data_df.empty:
        print('No songs, finishing execution')
        return False

In [79]:
#This is the load are of the data

engine = sqlalchemy.create_engine(DATABASE_LOCATION)
conn = sqlite3.connect('my_played_tracks.sqlite')
cursor = conn.cursor()

sql_query = """
CREATE TABLE IF NOT EXISTS my_played_tracks(
    played_at VARCHAR(200),
    track_name VARCHAR(200),
    artist VARCHAR(200),
    popularity INT,
)
"""

try:
    data_df.to_sql('my_played_tracks',engine,index=False, if_exists='append')
except:
    print("Data already exist in the database")

conn.close()
print("Close database successfully ")


Close database successfully 


In [7]:
new_token = "BQDKgpv3HNiOXPBsIXK2daD3KJalf-ko9dURVZb3BrE4GcbI92NgSxHXzJgpPS7pJ_W3n_y6WuZAaA7mGpk7zEOAGhx4LzvWjQzbDuudn37oU0A_rCK8Ccrm309wnQQmKmtvrfuUqyXUOIBUhD8IIJRRohTLawlYqWq7CGM0kF5d3aBzA9QgTtvtCvwkRurd"


headers = {
    "Accept" : "application/json",
    "Content-Type":"application/json",
    "Authorization":"Bearer {token}".format(token=new_token)
}

new_request = requests.get("https://api.spotify.com/v1/audio-features?ids=347H4auwVHcnWIsiYlVGWO,347H4auwVHcnWIsiYlVGWO",headers=headers)
new_request_data = new_request.json()
new_request_data

{'audio_features': [{'danceability': 0.444,
   'energy': 0.315,
   'key': 11,
   'loudness': -14.393,
   'mode': 1,
   'speechiness': 0.0485,
   'acousticness': 0.224,
   'instrumentalness': 6.07e-05,
   'liveness': 0.358,
   'valence': 0.378,
   'tempo': 120.663,
   'type': 'audio_features',
   'id': '347H4auwVHcnWIsiYlVGWO',
   'uri': 'spotify:track:347H4auwVHcnWIsiYlVGWO',
   'track_href': 'https://api.spotify.com/v1/tracks/347H4auwVHcnWIsiYlVGWO',
   'analysis_url': 'https://api.spotify.com/v1/audio-analysis/347H4auwVHcnWIsiYlVGWO',
   'duration_ms': 288573,
   'time_signature': 4},
  {'danceability': 0.444,
   'energy': 0.315,
   'key': 11,
   'loudness': -14.393,
   'mode': 1,
   'speechiness': 0.0485,
   'acousticness': 0.224,
   'instrumentalness': 6.07e-05,
   'liveness': 0.358,
   'valence': 0.378,
   'tempo': 120.663,
   'type': 'audio_features',
   'id': '347H4auwVHcnWIsiYlVGWO',
   'uri': 'spotify:track:347H4auwVHcnWIsiYlVGWO',
   'track_href': 'https://api.spotify.com/v1/

In [ ]:
r = requests.get("https://api.spotify.com/v1/me/player/recently-played?limit=50&after={time}".format(\
    time=yesterday_unix_timestamp), headers=headers)

data = r.json()
data_df = pd.json_normalize(data['items'])